## URL 구조 

scheme://사용자이름:비밀번호@호스트:포트/경로?질의#fragment

URL 허용문자가 있다. 즉 안전한 전송을 위해서 초기 URL 설계자들이 알파벳(ASCII) 만으로 URL을 작성하도록 하였다. 그런데, 점점 커지면서 ASCII 외의 문자들을 이스케이프(escape)처리하였다. 즉 이스케이프 문자로 인코딩 함을 말한다. 
  예를들어 빈문자열은 "%20" 으로, ~ 는 "%7" 로 인코딩 된다. 
  
  즉 https://ko.wikipedia.org/wiki/위키 는 https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4 
  
  이렇게 인코딩된다. 
  
  
  
  ko.wikipedia.org는 서버가 된다. 원하는 파일은 

In [ ]:
'''
url string을 넣으면, 원하는 part에 맞는 string을 return한다. 
'''

def split_url(line, part):
    if line.startswith("http://"):
        line=line[7:]
    if line.startswith("https://"):
        line=line[8:]
    if line.startswith("ftp://"):
        line=line[6:]
    if line.startswith("www."):
        line = line[4:]
    # 가장 첫번째 / 는 primary domain과 path를 나눠주는 경계이다.     
    slash_pos = line.find('/')
    if slash_pos > 0 and slash_pos < len(line)-1: # line = "fsdfsdf/sdfsdfsd"
        primarydomain = line[:slash_pos]
        path_argument = line[slash_pos+1:]
        
        # path 부분이긴 한데, 이 부분 또한 / 로 세부적으로 나눠질 수가 있다. 
        path_argument_tokens = path_argument.split('/')
        
        # 마지막 부분을 제외하고, 그 이전까지의 string들은 join으로 합친다. (왜냐면 그냥 경로를 나타내는 것임) 
        pathtoken = "/".join(path_argument_tokens[:-1])
        
        # 마지막 부분은 path 이후의 부분과 구분하기 위해서 일부러 분리한다. 
        last_pathtoken = path_argument_tokens[-1]
        
        
        if len(path_argument_tokens) > 2 and last_pathtoken == '':
            pathtoken = "/".join(path_argument_tokens[:-2])
            last_pathtoken = path_argument_tokens[-2]
        question_pos = last_pathtoken.find('?')
        if question_pos != -1:
            argument = last_pathtoken[question_pos+1:]
            pathtoken = pathtoken + "/" + last_pathtoken[:question_pos]     
        else:
            argument = ""
            pathtoken = pathtoken + "/" + last_pathtoken          
        last_slash_pos = pathtoken.rfind('/')
        sub_dir = pathtoken[:last_slash_pos]
        filename = pathtoken[last_slash_pos+1:]
        file_last_dot_pos = filename.rfind('.')
        if file_last_dot_pos != -1:
            file_extension = filename[file_last_dot_pos+1:]
            filename = filename[:file_last_dot_pos]
        else:
            file_extension = "" 
    elif slash_pos == 0:    # line = "/fsdfsdfsdfsdfsd"
        primarydomain = line[1:]
        pathtoken = ""
        argument = ""
        sub_dir = ""
        filename = ""
        file_extension = ""
    elif slash_pos == len(line)-1:   # line = "fsdfsdfsdfsdfsd/"
        primarydomain = line[:-1]
        pathtoken = ""
        argument = ""
        sub_dir = ""     
        filename = ""
        file_extension = ""
    else:      # line = "fsdfsdfsdfsdfsd"
        primarydomain = line
        pathtoken = ""
        argument = ""
        sub_dir = "" 
        filename = ""
        file_extension = ""
    if part == 'pd':
        return primarydomain
    elif part == 'path':
        return pathtoken
    elif part == 'argument': 
        return argument 
    elif part == 'sub_dir': 
        return sub_dir 
    elif part == 'filename': 
        return filename 
    elif part == 'fe': 
        return file_extension
    elif part == 'others': 
        if len(argument) > 0: 
            return pathtoken + '?' +  argument 
        else: 
            return pathtoken 
    else:
        return primarydomain, pathtoken, argument, sub_dir, filename, file_extension


In [2]:
path_argument = "path/to/server"

path_argument_tokens = path_argument.split('/')
print(path_argument_tokens)

pathtoken = "/".join(path_argument_tokens[:-1])
print(pathtoken)        
last_pathtoken = path_argument_tokens[-1]
print(last_pathtoken)

['path', 'to', 'server']
path/to
server


In [1]:
## 이게 뭐하는거지? 

def get_word_vocab(urls, max_length_words, min_word_freq=0): 
    # 즉 url들이 모여있는 url목록에서, 고유한 단어들을 모두 추출한다.
    # 단어들에 고유한 id를 보유한다. (보통 숫자형태로) 
    # 각각의 url들을 id로 표현한다. 이때, max_length_words보다 길이가 짧은 것들은 걍 0으로 채운다. 
    
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_length_words, min_frequency=min_word_freq) 
    start = time.time() 
    
    # 즉, 각 url들을 단어 ID, padding으로 치환한 다음, 그 치환된것을 np.array에 넣은것.
    x = np.array(list(vocab_processor.fit_transform(urls)))
    print("Finished build vocabulary and mapping to x in {}".format(time.time() - start))
    vocab_dict = vocab_processor.vocabulary_._mapping
    
    # key-value를 바꾼다. 원래는 단어가 key, 숫자가 value였는데, 숫자 id가 key, 단어가 value가 되게끔 바꾼다. 
    reverse_dict = dict(zip(vocab_dict.values(), vocab_dict.keys()))
    print("Size of word vocabulary: {}".format(len(reverse_dict)))
    return x, reverse_dict



In [4]:

from tensorflow import learn 
learn.preprocessing.VocabularyProcessor(test_sentence_len)

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [ ]:
## 위의 learn.preprocessing.VocabularyProcessor 는 deprecated  된단다. 

# 그래서 아래 코드로 대체하여 사용해보자 ? 
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(x_text)
x = tokenizer.texts_to_sequences(x_text)

x = tf.keraspreprocessing.sequence.pad_sequences(x, maxlen=max_document_length, padding='post', truncating='post')

In [2]:

# 여기에 들어가는 x는 URL이 숫자 id로 표현된 np array이다. 
# 보통 <UNK>가 0에 mapping 된다. <UNK> 는 ID 0을 갖고, 거의 언급이 안되는 애들을 뜻하는듯?

# 이 함수의 return은 단어들과, 특수문자들은 구분하여... 
'''
https://naver.com 이라면 
https, :, /, /, naver, . , com 이런식으로 구분하는 것이다. 
'''
def get_words(x, reverse_dict, delimit_mode, urls=None): 
    
     processed_x = []
    
    # 각 url 마다 뒤에 빈 것들을 없애고, id가 아닌, string 값으로 있는것.
    if delimit_mode == 0: 
        for url in x: 
            # words는 걍 string 단어의 집합? 
            words = []
            for word_id in url: 
                if word_id != 0: 
                    words.append(reverse_dict[word_id])
                # 즉 해당 url이 모두 끝났다는 말임. 
                else: 
                    break
                    
            processed_x.append(words) 
    
    # 
    elif delimit_mode == 1: 
        for i in range(x.shape[0]):
            word_url = x[i]
            raw_url = urls[i]
            words = []
            for w in range(len(word_url)): 
                word_id = word_url[w]
                # 0이라는건, 이제 URL이 끝이다. 따라서. raw_url을 그냥 넣는다. 
                #(어차피 아무것도 없을거니깐) 그리고 break
                if word_id == 0: 
                    words.extend(list(raw_url))
                    break
                else: 
                    # 즉 끝이 아니라, 어떤 특정단어에 mapping이 되었다.     
                    word = reverse_dict[word_id]
                    idx = raw_url.index(word) 
                    special_chars = list(raw_url[0:idx])
                    words.extend(special_chars) 
                    words.append(word) 
                    # 그 일반 단어 그 다음부터 세게끔 해서.. 
                    raw_url = raw_url[idx+len(word):]
                    if w == len(word_url) - 1: 
                        words.extend(list(raw_url))
            processed_x.append(words)
    return processed_x 

In [5]:



urls, labels = read_data(FLAGS["data.data_dir"]) 

# FLAGS["data.max_len_words"] 는 URL 당 최개 가질 수 있는 단어의 갯수 

# x는 url들을 id의 모음으로 나타낸것, word_reverse_dict는 단어-id 매핑정보 
x, word_reverse_dict = get_word_vocab(urls, FLAGS["data.max_len_words"])

# 특수문자는 미포함하여, [https, naver, com] 이면 delimite code =1 
# 특수문자 포함할거면, [https, :, /, /, naver, ., com] 이런식으로 표현가능. 
word_x = get_words(x, word_reverse_dict, FLAGS["data.delimit_mode"], urls)


ngramed_id_x, ngrams_dict, worded_id_x, words_dict = ngram_id_x(word_x, FLAGS["data.max_len_subwords"], high_freq_words)

def ngram_id_x(word_x, max_len_subwords, high_freq_words=None):   
    char_ngram_len = 1
    all_ngrams = set() 
    ngramed_x = []
    all_words = set() s in each word in a URL. Each word is either truncated or padded with a `<PADDING>` character to reach this length.                                            
    worded_x = []
    counter = 0
    for url in word_x:
        if counter % 100000 == 0: 
            print("Processing #url {}".format(counter))
        counter += 1  
        url_in_ngrams = []
        url_in_words = []
        words = urls in each word in a URL. Each word is either truncated or padded with a `<PADDING>` character to reach this length.                                            
        for word in words:
            ngrams = get_char_ngrams(char_ngram_len, word) 
            if (len(ngrams) > max_len_subwords) or \
                (high_freq_words is not None and len(word)>1 and not is_in(high_freq_words, word)):  
                all_ngrams.update(ngrams[:max_len_subwords])
                url_in_ngrams.append(ngrams[:max_len_subwords]) 
                all_words.add("<UNKNOWN>")
                url_in_words.append("<UNKNOWN>")
            else:     
                all_ngrams.update(ngrams)
                url_in_ngrams.append(ngrams) 
                all_words.add(word) 
                url_in_words.append(word) 
        ngramed_x.append(url_in_ngrams)
        worded_x.append(url_in_words) 

    all_ngrams = list(all_ngrams) 
    ngrams_dict = dict()
    for i in range(len(all_ngrams)):  
        ngrams_dict[all_ngrams[i]] = i+1 # ngram id=0 is for padding ngram   
    print("Size of ngram vocabulary: {}".format(len(ngrams_dict))) 
    all_words = list(all_words) 
    words_dict = dict() 
    for i in range(len(all_words)): 
        words_dict[all_words[i]] = i+1 #word id=0 for padding word 
    print("Size of word vocabulary: {}".format(len(words_dict)))
    print("Index of <UNKNOWN> word: {}".format(words_dict["<UNKNOWN>"]))        

    ngramed_id_x = []
    for ngramed_url in ngramed_x: 
        url_in_ngrams = []
        for ngramed_word in ngramed_url: 
            ngram_ids = [ngrams_dict[x] for x in ngramed_word] 
            url_in_ngrams.append(ngram_ids) 
        ngramed_id_x.append(url_in_ngrams)  
    worded_id_x = []
    for worded_url in worded_x: 
        word_ids = [words_dict[x] for x in worded_url]
        worded_id_x.append(word_ids) 
    
    return ngramed_id_x, ngrams_dict, worded_id_x, words_dict 